In [ ]:
import numpy as np
from pyannote.core import Segment, Timeline
import os
from sklearn.cluster import KMeans
import pandas as pd
import json
import pyannote.core.json
from numpy import genfromtxt
from sklearn.cluster import KMeans
from pydub import AudioSegment
from pydub.playback import play

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/Audio/Embeddings/Meal/Timestamps' )
file_names=sorted([i for i in os.listdir(".") if not i.startswith(".")])

In [ ]:
# import the complete file timestamps (from 0 to file end)
json_timelines=list()
for file in file_names:
    json_timelines.append(pyannote.core.json.load_from(file))

In [ ]:
# import the embedding vectors 
os.chdir( '/Users/andrei-macpro/Documents/Data/Audio/Embeddings/Meal/Embeddings' )
file_names=sorted([i for i in os.listdir(".") if not i.startswith(".")])
embeddings=list()
for file in file_names:
    embeddings.append(genfromtxt(file, delimiter=','))

In [ ]:
# run k means 
kmeans = KMeans(n_clusters=4)
predictions = list()
for file in embeddings:       
    labels = kmeans.fit_predict(file)
    predictions.append(labels)

In [ ]:
# tuple of 1-s second and its corresponding label 
list_of_tuples=[]
temp_list=[]
for prediction, timeline in zip(predictions, json_timelines):
    for label, segment in zip(prediction, timeline):
        temp_list.append(((label,segment)))
    list_of_tuples.append(temp_list)
    temp_list=[]
    

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/Audio/overlapping_speech/no_os_meal')  #if string starts with slash it is considered absolute
file_names=sorted([i for i in os.listdir(".") if not i.startswith(".")])

In [ ]:
# make new timelines from clusters 
def timelines_from_clusters(list_of_tuples,cluster_no):
    timelines_from_clusters = []
    for file in list_of_tuples:
        temp_list=[]
        for cluster_seg in file:
            if cluster_seg[0]==cluster_no:
                temp_list.append(cluster_seg[1])
        timelines_from_clusters.append(temp_list)
        
    return timelines_from_clusters


In [ ]:
clusters_0 = timelines_from_clusters(list_of_tuples,0)
clusters_1 = timelines_from_clusters(list_of_tuples,1)
clusters_2 = timelines_from_clusters(list_of_tuples,2)
clusters_3 = timelines_from_clusters(list_of_tuples,3)


In [ ]:
clusters_0


In [ ]:
# timelines with the 0-label 
timelines_0=[]
for time, file_name in zip(clusters_0, file_names):
    segments=[segment for segment in time]
    timeline=Timeline(segments=segments, uri=file_name)
    timelines_0.append(timeline)
    


In [ ]:
timelines_1=[]
for time, file_name in zip(clusters_1, file_names):
    segments=[segment for segment in time]
    timeline=Timeline(segments=segments, uri=file_name)
    timelines_1.append(timeline)
    
timelines_2=[]
for time, file_name in zip(clusters_2, file_names):
    segments=[segment for segment in time]
    timeline=Timeline(segments=segments, uri=file_name)
    timelines_2.append(timeline)
    
timelines_3=[]
for time, file_name in zip(clusters_3, file_names):
    segments=[segment for segment in time]
    timeline=Timeline(segments=segments, uri=file_name)
    timelines_3.append(timeline)

In [ ]:
# let's listen to these clusters with pydub 
os.chdir('/Users/andrei-macpro/Documents/Data/Audio/overlapping_speech/no_os_play')  #if string starts with slash it is considered absolute
file_names=sorted([i for i in os.listdir(".") if not i.startswith(".")])

In [ ]:
files_pydub = [AudioSegment.from_wav(file) for file in file_names]

In [ ]:
# function that writes to disk the timelines_from_clusters (i.e. for each recording the segments corresponding to the labels)
def process_files(timestamps, pydub_files, file_names):
    #timestamps = list containing timeline for every single file
    #pydub files = the original files in pydub format 
    list_pydub=list()
    speech=list()
    for timeline, pydub_file in zip(timestamps, pydub_files):
        speech=list()
        for key, value in dict(timeline).items():
            speech.append(pydub_file[key*1000:value*1000])
        list_pydub.append(speech)
    for file_name, audio in zip(file_names, list_pydub):
        sum(audio).export(file_name, format='wav')
        

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/Audio/Embeddings/Play/clusters/clusters_3')

In [ ]:
process_files(timelines_3, files_pydub, file_names)

In [ ]:
for segment in timelines_1: # cg
    play(file[segment.start*1000:segment.end*1000])

In [ ]:
for segment in timelines_2[1]: #noise
    play(file[segment.start*1000:segment.end*1000])

In [ ]:
for segment in timelines_3[18]: #bb
    play(file[segment.start*1000:segment.end*1000])

In [ ]:
# save timeline_from_cluster to disk 
for timeline in timelines_0:
    pyannote.core.json.dump_to(timeline, timeline.uri+".json")

In [ ]:
for timeline in timelines_1:
    pyannote.core.json.dump_to(timeline, timeline.uri+".json")

In [ ]:
for timeline in timelines_2:
    pyannote.core.json.dump_to(timeline, timeline.uri+".json")

In [ ]:
for timeline in timelines_3:
    pyannote.core.json.dump_to(timeline, timeline.uri+".json")